# VQE on Azure with Circuit Decomposition

Using strategy with both circuit and gate decomposition with no classical communication. VQE on Azure using sessions.

Prerequisites: 

1. Create a new Conda environment
```shell
conda create --name azure-knitting python=3.9.17
conda activate azure-knitting
```

1. Install everything needed to run the `modified-circuit-knitting-toolbox` locally: 
```shell
cd modified-circuit-knitting-toolbox
python -m pip install -e .
python -m pip install 'docplex>=2.23.222' 'cplex>=22.1.0.0' 'gurobipy==11.0.0'
```

1. Install everything needed to run on Azure:
```shell
python -m pip install --upgrade 'azure-quantum[qiskit]==1.2.1'
# python -m pip install qiskit-aer==0.12.1
```

### Documentation
Running on Azure: [instructions](https://learn.microsoft.com/en-us/azure/quantum/quickstart-microsoft-qiskit?tabs=tabid-quantinuum&pivots=platform-local#prerequisites)


First, enable communication with the instance

In [6]:
# Prevent overriding already run experiments
# exit()

In [7]:
# All imports
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider
from qiskit.circuit.library import ExcitationPreserving
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.optimizers import COBYLA

from circuit_knitting.cutting.gate_and_wire_cutting.algorithms.cut_vqe import CutVQE

In [8]:
# Set up the azure backend
workspace = Workspace(
    resource_id="/subscriptions/4ed1f7fd-7d9e-4c61-9fac-521649937e65/resourceGroups/Cutting/providers/Microsoft.Quantum/Workspaces/Cutting",
    location="eastus")
provider = AzureQuantumProvider(workspace)
# Using syntax checker to not use credits
backend = provider.get_backend("quantinuum.sim.h1-1sc")

usable_backends = ['quantinuum.sim.h1-1sc', 'quantinuum.sim.h1-2sc', 'quantinuum.sim.h1-1e', 'quantinuum.sim.h1-2e',
                   'quantinuum.qpu.h1-1', 'quantinuum.qpu.h1-2']

Problem: VQE for H2 molecule

In [9]:
# Set up the observables
H2_op_list = [('II', -1.0523732457728596),
              ('IZ', 0.39793742484317934),
              ('XX', 0.18093119978423144),
              ('ZI', -0.39793742484317934),
              ('ZZ', -0.011280104256235268)]
# Problem specification. Hamiltonian for H2 at 0.735A interatomic distance.
H2_op = SparsePauliOp.from_list(H2_op_list)


In [10]:
# Create ansatz and immediately decompose into individual gates
num_qubits = 2
circuit = ExcitationPreserving(num_qubits, flatten=True, reps=2, skip_final_rotation_layer=True).decompose(reps=2)

# Check how many parameters are in the ansatz
num_params = circuit.num_parameters

# Extract the pauli operators from the operations lit
observables = H2_op.paulis

In [11]:
# Callback function to check VQE process as it runs
def callback(eval_count, parameters, mean, std):
    print(f"Round num: {eval_count}, energy: {mean}, parameters: {parameters}")

In [12]:
# Define  estimator and optimizer
estimator = Estimator()
optimizer = COBYLA(maxiter=2)  # 2 Iteration for testing

# Set up VQE
vqe = CutVQE(estimator, circuit, optimizer, observables=observables, shots=2 ** 12, max_subcircuit_width=1, max_cuts=9,
             num_subcircuits=[2], model='gurobi', num_samples=1500, callback=callback, backend='azure',
             azure_backend=backend)

Set parameter Username


Academic license - for non-commercial use only - expires 2025-01-10
Set parameter TimeLimit to value 300
Set parameter Cutoff to value 1e+100
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.0.0 23A344)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 213 rows, 91 columns and 718 nonzeros
Model fingerprint: 0x04f4d5e8
Variable types: 0 continuous, 91 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 8.0000000
Presolve removed 213 rows and 91 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 8 

Optimal solution found (tolerance 1.00e-04)
Be

In [13]:
# Run VQE on Azure. Sessions are used for hybrid algorithms
with backend.open_session(name="Qiskit Session") as session:
    result = vqe.compute_minimum_eigenvalue(H2_op)

.................................................................................Round num: 1, energy: -1447.8850069391372, parameters: [ 3.93877709 -0.0721755   1.6750873  -2.77310351 -2.47283487  0.15102425]
...........................................................................Round num: 2, energy: 2895.7700138782743, parameters: [ 4.93877709 -0.0721755   1.6750873  -2.77310351 -2.47283487  0.15102425]


In [14]:
print(result)

{   'aux_operator_eigenvalues': None,
    'aux_operators_evaluated': None,
    'cost_function_evals': 2,
    'eigenstate': None,
    'eigenvalue': -1447.8850069391372,
    'optimal_circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f7c4e1c2ca0>,
    'optimal_parameters': {   ParameterVectorElement(θ[5]): 0.15102424787371582,
                              ParameterVectorElement(θ[4]): -2.4728348707159165,
                              ParameterVectorElement(θ[3]): -2.7731035081896396,
                              ParameterVectorElement(θ[2]): 1.6750872952801519,
                              ParameterVectorElement(θ[1]): -0.07217549796682388,
                              ParameterVectorElement(θ[0]): 3.9387770884841586},
    'optimal_point': array([ 3.93877709, -0.0721755 ,  1.6750873 , -2.77310351, -2.47283487,
        0.15102425]),
    'optimal_value': -1447.8850069391372,
    'optimizer_evals': None,
    'optimizer_result': <qiskit_algorithms.optimizers.optimizer.